### Apellidos y Nombres:

Lettere Dragosavljevich Mathias Giuseppe

### Fecha:

05-09-2023

# **Introduction to SparkSQL**


This lab goes over the basic operations of Apache SparkSQL.


***


## Google Colab Setup

If you are going to use Google Colab instead of a Spark Cluster, you will need to run the following code to install Apache Spark.

In [22]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [23]:
#If the following links don't work, you will have to update them with the last versions of Apache Spark
!wget -q https://dlcdn.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar xf spark-3.4.1-bin-hadoop3.tgz

In [24]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

## Setup


In [25]:
# Installing required packages
!pip install pyspark
!pip install findspark
!pip install pyarrow
!pip install pandas
!pip install numpy
!pip install random

ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random


In [26]:
import findspark
findspark.init()

In [27]:
import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [28]:
import random

## Spark session


#### Spark session and context


In [29]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

ValueError: ignored

#### Initialize Spark session



In [30]:
spark

## Media


In [31]:
from pyspark.rdd import RDD

rdd = sc.parallelize(random.sample(range(5, 300), 100))

def media(d):
  sum = d.sum()
  n = d.count()
  return sum/float(n)

print("La madia del RDD es: ", media(rdd))

La madia del RDD es:  164.86


## Mediana


In [37]:
def mediana(d):
  sortear = d.sortBy(lambda x:x)
  indexar = sortear.zipWithIndex().map(lambda value_key : (value_key[1], value_key[0]))
  n = d.count()

  if(n%2 == 1):
    MIndex = (n-1)/2
    return indexar.filter(lambda key_value : key_value[0] == MIndex).collect()[0][1]
  else:
    i1 = (n/2)-1
    i2 = n/2

    v1 = indexar.filter(lambda key_value : key_value[0] == i1).collect()[0][1]
    v2 = indexar.filter(lambda key_value : key_value[0] == i2).collect()[0][1]
    return (v1+v2)/2

print("La mediana del RDD es: ", mediana(rdd))

La mediana del RDD es:  176.0


## Desviación Estándar

In [44]:
from math import sqrt
def stdDev(d):
  m = media(d)
  n = d.count()
  return sqrt(d.map(lambda x: pow(x - m, 2)).sum()/float(n))

#m = sqrt(rdd.map(lambda x: pow(x - media(d), 2).sum()/float(rdd.count))
print("La desviación estándar del RDD es: ", stdDev(rdd))


La desviación estándar del RDD es:  83.20505032748915


## Skewness

In [45]:
def skw(d):
  m = media(d)
  sd = stdDev(d)
  n = d.count()
  return (1/n)*d.map(lambda x: pow(x-m, 3)/pow(sd,3)).sum()

print("La Skewdness del RDD es: ", skw(rdd))

La Skewdness del RDD es:  -0.2859177847201621


## Kurtosis

In [46]:
def kurt(d):
  m = media(d)
  sd = stdDev(d)
  n = d.count()
  return (1/n)*d.map(lambda x: pow(x-m, 4)/pow(sd,4)).sum()

print("La Kurtosis del RDD es: ", kurt(rdd))

La Kurtosis del RDD es:  1.9711548070784601


## Covarianza

In [47]:
rdd2 = sc.parallelize(random.sample(range(5, 300), 100))

def cov(x, y):
  rddXY = x.zip(y)
  mx= media(x)
  my= media(y)
  return rddXY.map(lambda x_y: (x_y[0] - mx)*(x_y[1]-my)).sum()/rddXY.count()

print("La Covarianza entre X y Y es: ", cov(rdd, rdd2))

La Covarianza entre X y Y es:  1396.9624


## Correlación

In [49]:
def correl(x, y):
  covXY = cov(x, y)
  sdx = stdDev(x)
  sdy = stdDev(y)
  return covXY/(sdx*sdy)

print("La Correlación entre X y Y es: ", correl(rdd, rdd2))

La Correlación entre X y Y es:  0.2035579950399146
